In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!ls drive

'My Drive'


In [0]:
train=pd.read_csv('drive/My Drive/Colab Notebooks/train.tsv',sep='\t')

In [4]:
!pip install wordbatch

    100% |████████████████████████████████| 1.2MB 16.9MB/s 
    100% |████████████████████████████████| 51kB 23.8MB/s 
    100% |████████████████████████████████| 112kB 33.5MB/s 
    100% |████████████████████████████████| 1.9MB 16.9MB/s 
  Running setup.py bdist_wheel for wordbatch ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/68/c1/2d/f2ef7f03fd110866c2836eb3bf0c6790a218a01e1a3b652cbc
  Running setup.py bdist_wheel for python-Levenshtein ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Running setup.py bdist_wheel for py-lz4framed ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/55/96/0b/b8e76112de5c48181d1e9318c5d86192947f692f7f8972fc6d
Successfully built wordbatch python-Levenshtein py-lz4framed


In [0]:
import wordbatch

from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL, FM_FTRL

from nltk.corpus import stopwords
import re

In [0]:
NUM_BRANDS = 4500
NUM_CATEGORIES = 1250
#nrow_train = train.shape[0]
y = np.log1p(train["price"])


In [0]:
def category(category_name):
    try:
        main, sub1, sub2= category_name.split('/')
        return main, sub1, sub2
    except:
        return np.nan, np.nan, np.nan
train['category_main'], train['category_sub1'], train['category_sub2'] = zip(*train['category_name'].apply(category))

In [0]:
train['category_main'].fillna(value='missing', inplace=True)
train['category_sub1'].fillna(value='missing', inplace=True)
train['category_sub2'].fillna(value='missing', inplace=True)
train['brand_name'].fillna(value='missing', inplace=True)
train['item_description'].fillna(value='missing', inplace=True)


In [10]:
import nltk
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def cl_html(s):
    c1 = re.compile('<.*?>')
    c2 = re.sub(c1, ' ', str(s))
    return c2
def cl_p(s): 
    cl = re.sub(r'[?|!|\'|"|#]',r'',s)
    cl = re.sub(r'[.|,|)|(|\|/]',r' ',cl)
    cl = cl.strip()
    cl = cl.replace("\n"," ")
    return cl
stop_words = set(stopwords.words('english'))

sw= re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def rem_stop(s):
    global sw
    return sw.sub(" ", s)
train['item_description']=train['item_description'].apply(rem_stop)
train['item_description']=train['item_description'].apply(cl_p)
train['item_description']=train['item_description'].apply(cl_html)

In [0]:
b= train['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
train.loc[~train['brand_name'].isin(b), 'brand_name'] = 'missing'
cat1 = train['category_main'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
cat2= train['category_sub1'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
cat3 = train['category_sub2'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
train.loc[~train['category_main'].isin(cat1), 'category_main'] = 'missing'
train.loc[~train['category_sub1'].isin(cat2), 'category_sub1'] = 'missing'
train.loc[~train['category_sub2'].isin(cat3), 'category_sub2'] = 'missing'


In [0]:
train['category_main'] = train['category_main'].astype('category')
train['category_sub1'] = train['category_sub1'].astype('category')
train['category_sub2'] = train['category_sub2'].astype('category')
train['item_condition_id'] = train['item_condition_id'].astype('category')


In [17]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_main,category_sub1,category_sub2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,missing,10.0,1,description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,keyboard great condition works like came ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top hint lace key hole back p...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,missing,35.0,1,New tags Leather horses Retail [rm] Stan...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,missing,44.0,0,Complete certificate authenticity,Women,Jewelry,Necklaces


In [0]:
train.drop('category_name', axis=1, inplace=True)

In [0]:
wb = wordbatch.WordBatch( extractor=(WordBag, {"hash_ngrams": 2, "hash_ngrams_weights": [1.5, 1.0],
                                                                  "hash_size": 2 ** 29, "norm": None, "tf": 'binary',
                                                                  "idf": None,
                                                                  }), procs=8)

In [25]:
wb.dictionary_freeze= True
nm = wb.fit_transform(train['name'])


Normalize text


In [0]:

nm = nm[:, np.where(nm.getnnz(axis=0) > 1)[0]]


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [0]:
wb = CountVectorizer()
c1 = wb.fit_transform(train['category_main'])
c2 = wb.fit_transform(train['category_sub1'])
c3 = wb.fit_transform(train['category_sub2'])


In [0]:
wb = wordbatch.WordBatch(extractor=(WordBag, {"hash_ngrams": 2, "hash_ngrams_weights": [1.0, 1.0],
                                                                  "hash_size": 2 ** 28, "norm": "l2", "tf": 1.0,
                                                                  "idf": None})
                             , procs=8)

In [30]:
wb.dictionary_freeze= True
descp = wb.fit_transform(train['item_description'])



Normalize text


In [0]:

descp = descp[:, np.where(descp.getnnz(axis=0) > 1)[0]]




In [0]:
lb = LabelBinarizer(sparse_output=True)
br = lb.fit_transform(train['brand_name'])



In [0]:
from scipy.sparse import csr_matrix, hstack

In [0]:
dum = csr_matrix(pd.get_dummies(train[['item_condition_id', 'shipping']],
                                          sparse=True).values)



In [0]:
mer = hstack((dum, descp, br, c1, c2, c3, nm)).tocsr()

In [36]:
mer.shape

(1482535, 1969749)

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
X_train, X_test, y_train, y_test = train_test_split(mer, y, test_size=0.3)

In [0]:
from sklearn.metrics.scorer import make_scorer
import math

In [0]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    
    #to_sum = [math.(y_pred[i]  - math.(y[i]) )** 2.0 for i,pred in enumerate(y_pred)]
    #return (sum(to_sum) * (1.0/len(y))) ** 0.5
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

In [0]:
from sklearn.linear_model import Ridge

In [43]:
ridge = Ridge(solver='auto', fit_intercept=True, alpha=0.4, max_iter=200, normalize=False, tol=0.01)
ridge.fit(X_train, y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=200,
   normalize=False, random_state=None, solver='auto', tol=0.01)

In [0]:
r1=ridge.predict(X_test)

In [0]:
t1=y_test.values

In [50]:
e1=rmsle(np.expm1(t1), np.expm1(r1))
print(e1)

0.4619292564688951


In [0]:
model = FTRL(alpha=0.01, beta=0.1, L1=0.00001, L2=1.0, D=mer.shape[1], iters=50, inv_link="identity", threads=1)

In [53]:
model.fit(X_train, y_train)

Total e: 403428.8855703045
Total e: 360119.51955482713
Total e: 346929.083079405
Total e: 338236.73601981264
Total e: 331647.51070727676
Total e: 326310.30801500723
Total e: 321808.58222572703
Total e: 317905.8135642228
Total e: 314452.8854546926
Total e: 311353.3561254611
Total e: 308538.24766812066
Total e: 305959.141270538
Total e: 303577.4596535562
Total e: 301363.11967425636
Total e: 299293.58012672473
Total e: 297350.8247406983
Total e: 295519.42397617735
Total e: 293786.74304149585
Total e: 292141.55867052154
Total e: 290575.50307418336
Total e: 289081.25113448274
Total e: 287652.8727448411
Total e: 286284.20580122387
Total e: 284969.97423316573
Total e: 283706.45108065655
Total e: 282489.50057051127
Total e: 281315.70314124174
Total e: 280182.2661530445
Total e: 279086.2214007553
Total e: 278025.1910019317
Total e: 276996.8630962024
Total e: 275999.45017437247
Total e: 275030.9989852259
Total e: 274089.9316866973
Total e: 273174.2861399264
Total e: 272283.04525211744
Total e: 2

In [0]:
m1=model.predict(X_test)

In [0]:
e2=rmsle(np.expm1(t1), np.expm1(m1))


In [68]:
print(e2)

0.4408043784945176


In [0]:
 model2 = FM_FTRL( L1=0.00001, L2=1.0,D=mer.shape[1],
                    inv_link="identity")


In [57]:
model2.fit(X_train, y_train)

Total e: 365206.91267058894
Total e: 310269.948716321
Total e: 289058.5687083469
Total e: 274074.3341902569
Total e: 262175.3378282672


In [0]:
m2 = model2.predict(X_test)

In [0]:
e3=rmsle(np.expm1(t1), np.expm1(m2))


In [65]:
print(e3)

0.4289315442046666


In [70]:
modelDF = pd.DataFrame({
    'Model'       : ['Ridge Regression', 'FTRL', 'FM-FTRL'],
    
    'RMLSE'   : [ e1, e2,e3],
    
    }, columns = ['Model', 'RMLSE'])

modelDF.sort_values(by='RMLSE', ascending=False)

,Model,RMLSE
0,Ridge Regression,0.461929
1,FTRL,0.440804
2,FM-FTRL,0.428932
